## Comments列の特徴量を作成する

In [2]:
import pandas as pd
import scipy as sc
import numpy as np
import sklearn
import pickle
import pathlib as Path
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sb
sb.set(font='IPAexGothic')

import multiprocessing
import itertools
import collections
import datetime
import gc

from tqdm.notebook import tqdm

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

data_path = Path.Path("../data")
img_path = Path.Path("../img")
result_path = Path.Path("../result")

prefix = 'ana004'

## データ読み込み

In [3]:
train_df = pd.read_pickle(data_path / 'train_df.pkl')

train_df

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,doi_cites,cites
0,hep-ph/9902295,Michael Kraemer,Mark E. Hayes (University College London) and ...,Heavy-Flavour Production at HERA,"LaTeX, 21 pages, 13 Postscript figures. Summar...","J.Phys.G25:1477-1493,1999",10.1088/0954-3899/25/7/332,"CERN-TH/99-30, UCL/HEP 99-03",hep-ph hep-ex,None,We review the theoretical and experimental s...,"[[{'version': 'v1', 'created': 'Wed, 10 Feb 19...",2008-11-26,"[[Hayes, Mark E., , University College London]...",1,NaN
1,1403.7138,Aigen Li,"Qi Li, S.L. Liang, Aigen Li (University of Mis...",Spectropolarimetric Constraints on the Nature ...,"5 pages, 2 figures; accepted for publication i...",None,10.1093/mnrasl/slu021,None,astro-ph.GA,http://arxiv.org/licenses/nonexclusive-distrib...,While it is well recognized that interstella...,"[[{'version': 'v1', 'created': 'Thu, 27 Mar 20...",2015-06-19,"[[Li, Qi, , University of Missouri], [Liang, S...",8,7.0
2,1405.5857,Michael Mortonson,"Michael J. Mortonson, Uro\v{s} Seljak",A joint analysis of Planck and BICEP2 B modes ...,"13 pages, 4 figures; submitted to JCAP; refere...",JCAP10(2014)035,10.1088/1475-7516/2014/10/035,None,astro-ph.CO gr-qc hep-ph hep-th,http://arxiv.org/licenses/nonexclusive-distrib...,We analyze BICEP2 and Planck data using a mo...,"[[{'version': 'v1', 'created': 'Thu, 22 May 20...",2014-10-17,"[[Mortonson, Michael J., ], [Seljak, Uroš, ]]",122,188.0
3,1807.01034,Evangelos Thomas Karamatskos,"Evangelos T. Karamatskos, Sebastian Raabe, Ter...",Molecular movie of ultrafast coherent rotation...,9 Figures,"Nat Commun 10, 3364 (2019)",10.1038/s41467-019-11122-y,None,physics.chem-ph physics.atom-ph quant-ph,http://arxiv.org/licenses/nonexclusive-distrib...,Recording molecular movies on ultrafast time...,"[[{'version': 'v1', 'created': 'Tue, 3 Jul 201...",2020-05-19,"[[Karamatskos, Evangelos T., ], [Raabe, Sebast...",6,8.0
4,1905.05921,Juanjuan Gu,Juanjuan Gu and Yun Jing,A Modified Mixed Domain Method for Modeling Ac...,None,None,10.1121/10.0001454,None,physics.med-ph physics.comp-ph,http://arxiv.org/licenses/nonexclusive-distrib...,"In this paper, phase correction and amplitud...","[[{'version': 'v1', 'created': 'Wed, 15 May 20...",2020-07-15,"[[Gu, Juanjuan, ], [Jing, Yun, ]]",0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851519,1708.06097,Haim Diamant,Rony Granek and Haim Diamant,Membrane undulations in a structured fluid: Un...,10 pages,"Eur. Phys. J. E 41, 1 (2018)",10.1140/epje/i2018-11607-x,None,cond-mat.soft,http://arxiv.org/licenses/nonexclusive-distrib...,The dynamics of membrane undulations inside ...,"[[{'version': 'v1', 'created': 'Mon, 21 Aug 20...",2018-02-20,"[[Granek, Rony, ], [Diamant, Haim, ]]",0,NaN
851520,1904.07627,Chonglong Liu,"C. L. Liu, Xiao-Dong Yu, D. M. Tong",Flag Additivity in Quantum Resource Theories,6 pages,"Phys. Rev. A 99, 042322(2019)",10.1103/PhysRevA.99.042322,None,quant-ph,http://arxiv.org/licenses/nonexclusive-distrib...,Quantum resource theories offer a powerful f...,"[[{'version': 'v1', 'created': 'Tue, 16 Apr 20...",2019-04-17,"[[Liu, C. L., ], [Yu, Xiao-Dong, ], [Tong, D. ...",2,NaN
851521,1507.01140,Othman Benomar,"O. Benomar, M. Takata, H. Shibahashi, T. Ceill...",Nearly-uniform internal rotation of solar-like...,Accepted to MNRAS,None,10.1093/mnras/stv1493,None,astro-ph.SR,http://arxiv.org/licenses/nonexclusive-distrib...,The rotation rates in the deep interior and ...,"[[{'version': 'v1', 'created': 'Sat, 4 Jul 201...",2015-08-06,"[[Benomar, O., ], [Takata, M., ], [Shibahashi,...",42,NaN
851522,2006.03674,Sukriti Manna,"Troy D Loeffler, Sukriti Manna, Tarak K Patra,...",Active Learning A Neural Network Model For Gol...,None,None,10.1002/cctc.202000774,None,physics.comp-ph cond-mat.mtrl-sci,http://arxiv.org/licenses/nonexclusive-distrib...,Small metal clusters are of fundamental scie...,"[[{'version': 'v1', 'created': 'Fri, 5 Jun 202..."

## 特徴量作成
> comments列から取得できる要素を精査

In [7]:
# 上から100行出してみる
list(train_df['comments'][:100])

["LaTeX, 21 pages, 13 Postscript figures. Summary of the working group\n  on `Heavy Flavour Physics' at the 3rd UK Phenomenology Workshop on HERA\n  Physics, Durham, UK, 20 -- 25 September 1998. To appear in the proceedings",
 '5 pages, 2 figures; accepted for publication in MNRAS Letters',
 '13 pages, 4 figures; submitted to JCAP; references added; minor\n  changes to clarify methodology in response to referee report; expanded\n  analysis (see new versions of Figs. 1 and 2) to include Planck measurement of\n  dust polarization power in the BICEP2 field from\n  http://arxiv.org/abs/1409.5738',
 '9 Figures',
 None,
 'This is a preprint of the original publication that is available at\n  https://link.springer.com/article/10.1007/s13171-018-00160-1',
 'revisions to ApJ ms first submitted 8/21/99; uses a higher and\n  flatter input spectrum, with modified implications suggesting preference for\n  funnel models',
 '13 pages, 8 figures, RevTeX. Final version to be published in\n  European Ph

> pages, figures, tablesの値を抽出してみる

In [243]:
import re

# 対象の数値を抽出する関数
def extract_nums(input_str, target_str):
    
    # 入力がNoneだった場合、nanを返却
    if input_str is None: return np.nan
    
    # 指定した単語の直前にある数字を抽出する
    find_list = re.findall(r'.*[^\d](\d+).*?%s' % target_str, ' ' + input_str)
    
    # 抽出できた場合、最初の値をintに直して返却する 抽出できなかった場合、nanを返却
    return int(find_list[0]) if len(find_list) > 0 else np.nan

In [242]:
%%time

# 特徴量DFを用意
comment_x_df = pd.DataFrame()

# 抽出対象文字列定義
target_list = ['pages', 'figures', 'tables']

# それぞれ抽出した値を格納した列を作成、nanは0に直し全てを整数型とする
comment_x_df[[target for target in target_list]] = train_df['comments'].apply(lambda x: pd.Series([extract_nums(x, target) for target in target_list])).fillna(0).astype('Int64')

comment_x_df

Wall time: 5min 51s


,pages,figures,tables
0,21,13,0
1,5,2,0
2,13,4,0
3,0,0,0
4,0,0,0
...,...,...,...
851519,10,0,0
851520,6,0,0
851521,0,0,0
851522,0,0,0


In [251]:
# tablesの値もちゃんと入ってる？
comment_x_df[comment_x_df['tables'] > 0]

# tablesに値が入っているもののうち、冒頭を確認
train_df.iloc[comment_x_df[comment_x_df['tables'] > 0].index[0]]

,pages,figures,tables
33,32,2,5
37,57,13,5
59,8,5,2
74,0,0,2
75,10,2,3
...,...,...,...
851401,28,0,7
851407,6,9,2
851435,4,3,2
851472,14,5,2


id                                                 astro-ph/9812279
submitter                                              Toshiya Ueta
authors           Margaret Meixner, Toshiya Ueta (UIUC), Aditya ...
title             A Mid-Infrared Imaging Survey of Proto-Planeta...
comments          32 pages, 5 tables, 2 e/ps figures (fig3 is av...
journal-ref                                    1999, ApJS, 122, 221
doi                                                  10.1086/313215
report-no                                                      None
categories                                                 astro-ph
license                                                        None
abstract            We present the data from a mid-infrared imag...
versions          [[{'version': 'v1', 'created': 'Tue, 15 Dec 19...
update_date                                              2009-10-31
authors_parsed    [[Meixner, Margaret, , UIUC], [Ueta, Toshiya, ...
doi_cites                                       

> 大丈夫そう

In [253]:
# 特徴量保存
comment_x_df.to_pickle(data_path / ('%s_x.pkl' % prefix))